In [1]:
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import numpy as np

import pandas as pd
# 모든 행을 출력하도록 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 기본값으로 설정 (처음 5개와 마지막 5개 행만 출력)
pd.reset_option('display.max_rows')

# 출력 포맷 설정 (소수점 4자리까지)
pd.options.display.float_format = '{:.4f}'.format

import platform
import seaborn as sns

import matplotlib.pyplot as plt

# 운영 체제 확인
if platform.system() == 'Darwin':  # Mac
    print('apple gothic')
    font_name = 'AppleGothic'
elif platform.system() == 'Windows':  # Windows
    font_name = 'NanumGothic'
else:
    font_name = None

# 한글 폰트 설정
if font_name:
    plt.rcParams['font.family'] = font_name

# 마이너스 부호 설정
plt.rcParams['axes.unicode_minus'] = False

apple gothic


## 데이터 로드

In [2]:
df_raw = pd.read_csv('../data/이자보상배율_KOSPI+KOSDAQ.csv', dtype = {'거래소코드' : 'object'}, encoding='cp949')

In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26623 entries, 0 to 26622
Data columns (total 17 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   회사명                                               26623 non-null  object 
 1   거래소코드                                             26623 non-null  object 
 2   회계년도                                              26623 non-null  object 
 3   상장일                                               26623 non-null  object 
 4   상장폐지일                                             1883 non-null   object 
 5   [제조]이자보상배율(이자비용)(IFRS연결)                          18466 non-null  float64
 6   [제조]이자보상배율(이자비용)(IFRS)                            23905 non-null  float64
 7   [U01B420000000][제조]* (정상)영업손익(보고서기재)(IFRS연결)(천원)  18613 non-null  float64
 8   [U01B470010000][제조]   이자비용(IFRS연결)(천원)            16359 non-null  float64
 9   [U01B550010000][제

In [4]:
# 정렬 후 저장
df_raw.sort_values(['거래소코드', '회계년도', '상장일'], inplace=True)
df_raw.reset_index(drop=True, inplace=True)
df_raw.shape

(26623, 17)

## 불필요 데이터 제거

### (1) 중복 행 제거

In [5]:
# (1) 중복 행 제거 (이전상장한 경우 : KOSDAQ -> KOSPI)

df_drop_dup = df_raw.drop_duplicates(
    subset = ['거래소코드', '회계년도'], keep='first'
)
df_drop_dup.reset_index(drop=True)

df_drop_dup.shape

(26543, 17)

### (2) 12월 외 결산 보고서

In [6]:
df_drop_dup['결산월'] = pd.to_datetime(df_drop_dup['회계년도']).dt.month
df_drop_dup['결산월'].value_counts()

결산월
12    26351
3        98
6        64
9        28
10        1
11        1
Name: count, dtype: int64

In [7]:
df_drop_dup[df_drop_dup['결산월']!=12]['거래소코드'].unique()

array(['000220', '000440', '000760', '000970', '002720', '003000',
       '005800', '005850', '006370', '007570', '011330', '018700',
       '019660', '024720', '024850', '025270', '032680', '032980',
       '033250', '034950', '035900', '039610', '040910', '042370',
       '048770', '054050', '054180', '054220', '066430', '082920',
       '092130', '114190', '121800', '124500', '131100', '160600',
       '161890', '184230', '192400', '227950', '244920', '263750',
       '278280', '330350', '950010'], dtype=object)

In [8]:
# (대부분) 결산월이 변경된 케이스 : 12월 외 -> 12월
df_drop_dup.loc[df_drop_dup['거래소코드']=='000220', ['회사명', '거래소코드', '회계년도']]

,회사명,거래소코드,회계년도
132,(주)유유제약,000220,2011/03
133,(주)유유제약,000220,2012/03
134,(주)유유제약,000220,2013/03
135,(주)유유제약,000220,2014/03
136,(주)유유제약,000220,2015/03
137,(주)유유제약,000220,2016/03
138,(주)유유제약,000220,2017/03
139,(주)유유제약,000220,2018/12
140,(주)유유제약,000220,2019/12
141,(주)유유제약,000220,2020/12


In [9]:
# 결산월이 변경된 케이스 : 12월 -> 12월 외

# 거래소코드 : 131100, 227950, 024850
df_drop_dup.loc[df_drop_dup['거래소코드']=='227950', ['회사명', '거래소코드', '회계년도']]

,회사명,거래소코드,회계년도
22792,(주)엔투텍,227950,2015/12
22793,(주)엔투텍,227950,2016/12
22794,(주)엔투텍,227950,2017/12
22795,(주)엔투텍,227950,2018/12
22796,(주)엔투텍,227950,2020/06
22797,(주)엔투텍,227950,2021/06
22798,(주)엔투텍,227950,2022/06


In [10]:
# 결산월 : 3월 -> 6월 -> 12월

# 거래소코드 : 042370, 082920
df_drop_dup.loc[df_drop_dup['거래소코드']=='082920', ['회사명', '거래소코드', '회계년도']]

,회사명,거래소코드,회계년도
15183,(주)비츠로셀,082920,2011/03
15184,(주)비츠로셀,082920,2012/03
15185,(주)비츠로셀,082920,2013/03
15186,(주)비츠로셀,082920,2014/03
15187,(주)비츠로셀,082920,2015/06
15188,(주)비츠로셀,082920,2016/06
15189,(주)비츠로셀,082920,2017/06
15190,(주)비츠로셀,082920,2018/12
15191,(주)비츠로셀,082920,2019/12
15192,(주)비츠로셀,082920,2020/12


In [11]:
# 결산월이 12월이 아닌 경우
# 거래소코드 : 033250(6월), 054220(3월)
df_drop_dup.loc[df_drop_dup['거래소코드']=='033250', ['회사명', '거래소코드', '회계년도']]

,회사명,거래소코드,회계년도
8124,(주)체시스,033250,2011/06
8125,(주)체시스,033250,2012/06
8126,(주)체시스,033250,2013/06
8127,(주)체시스,033250,2014/06
8128,(주)체시스,033250,2015/06
8129,(주)체시스,033250,2016/06
8130,(주)체시스,033250,2017/06
8131,(주)체시스,033250,2018/06
8132,(주)체시스,033250,2019/06
8133,(주)체시스,033250,2020/06


In [12]:
# 12월 결산, 특정 기간에만 결산월이 12월이 아닌 케이스
# -> 거래정지로 인해 일시적인 현상으로 파악
# 거래소코드 : 011330, 263750, 160600
df_drop_dup.loc[df_drop_dup['거래소코드']=='011330', ['회사명', '거래소코드', '회계년도', '상장폐지일']]

# 011330 : (주)유니켐
# 거래정지

# 160600 : (주)이큐셀
# 사업보고서 제출 기한 연장 신고 : 회계감사 관련 외부감사 진행중
# 제17기(2020년 01월 01일~2020년 06월 30일)의 회계연도에 대한 회계감사와 관련하여 현재 외부감사인으로부터 외부감사를 수행 중에 있습니다. 
# 재무제표 감사 범위 내 중요한 자료 등에 대한 수령 및 절차가 지연되고 있어 공시일 현재 감사절차가 완료되지 않은 상태입니다. 

,회사명,거래소코드,회계년도,상장폐지일
4770,(주)유니켐,011330,2011/12,NaN
4771,(주)유니켐,011330,2012/12,NaN
4772,(주)유니켐,011330,2013/12,NaN
4773,(주)유니켐,011330,2015/06,NaN
4774,(주)유니켐,011330,2016/12,NaN
4775,(주)유니켐,011330,2017/12,NaN
4776,(주)유니켐,011330,2018/12,NaN
4777,(주)유니켐,011330,2019/12,NaN
4778,(주)유니켐,011330,2020/12,NaN
4779,(주)유니켐,011330,2021/12,NaN


In [13]:
# 결산월이 12월이 아닌 데이터 제거
df_drop_month = df_drop_dup[df_drop_dup['결산월']==12]
df_drop_month.shape

(26351, 18)

In [14]:
df_drop_month['결산월'].value_counts()

결산월
12    26351
Name: count, dtype: int64

In [15]:
# 결산월 컬럼 제거
df_drop_month = df_drop_month.drop(columns=['결산월'])

### (3) 상장 이전 데이터 제거

In [16]:
df_drop_month['결산년도'] = pd.to_datetime(df_drop_month['회계년도']).dt.year
df_drop_month['상장년도'] = pd.to_datetime(df_drop_month['상장일']).dt.year

In [17]:
# 상장년도 이후 결산 보고서만 남기기
df_drop = df_drop_month[df_drop_month['결산년도']>=df_drop_month['상장년도']]
df_drop.shape

(22485, 19)

### (4) KONEX 상장 종목 제거

In [18]:
df = df_drop.copy()

In [19]:
# 회계년도별 상장 종목 리스트 조회
from pykrx import stock
from tqdm import tqdm

df['market']=np.nan

for year in tqdm(range(2011, 2023)):
    kospi = stock.get_market_ticker_list(f'{year}1231', market='KOSPI')
    df.loc[(df['결산년도']==year) & (df['거래소코드'].isin(kospi)), 'market'] = 'KOSPI'

    kosdaq = stock.get_market_ticker_list(f'{year}1231', market='KOSDAQ')
    df.loc[(df['결산년도']==year) & (df['거래소코드'].isin(kosdaq)), 'market'] = 'KOSDAQ'

    konex = stock.get_market_ticker_list(f'{year}1231', market='KONEX')
    df.loc[(df['결산년도']==year) & (df['거래소코드'].isin(konex)), 'market'] = 'KONEX'

df['market'].isna().sum()

100%|██████████| 12/12 [00:13<00:00,  1.15s/it]


0

In [20]:
# 시장별 상장 종목 수
df.groupby('market')['거래소코드'].nunique()

market
KONEX       82
KOSDAQ    1686
KOSPI      803
Name: 거래소코드, dtype: int64

In [21]:
# 코넥스 상장 종목 제거
df = df[df['market']!='KONEX']
df.shape

(22245, 20)

In [22]:
# 시장별 상장 종목 수
df.groupby('market')['거래소코드'].nunique()

market
KOSDAQ    1686
KOSPI      803
Name: 거래소코드, dtype: int64

In [23]:
df.to_csv('../data/1_불필요종목제거.csv', index=None)